# TP4
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Exercício 4.2

### Descrição do Problema



seq = [-2,1,2,-1,4,-4,-3,3]

changed = True

while changed:

    changed = False
    
    for i in range(len(seq) - 1):
    
        if seq[i] > seq[i+1]:
        
            seq[i], seq[i+1] = seq[i+1], seq[i]
            
            changed = True
            
pass  

### Abordagem do Problema

Para resolver todas as facetas deste problema será nesseçário:
1. Criar uma Pre condição do algoritmo.
2. Criar uma Pos condição do algoritmo.
3. Criar uma relação de transição que traduza o ciclo 'for'.
4. Provar a correção do algoritmo

1-
A pré condição do algoritmmo é definida pela seguinte expresão:
$$ (n \ge 0) \land (changed \leftrightarrow True) \land axioms \land store $$

2-
A pós condição do algoritmmo é definida pela seguinte expresão:
$$ (changed \leftrightarrow False) \land (\forall_i \quad 0 \le n \lt N-1 \rightarrow seq[i] \le seq[i+1])$$

3-
Para definir a relação de transição decidimos seguir dois caminhos diferrentes:
1. definir o ciclo for com o auxilio de uma função recursiva.
Essa função é tal que:

```python
f(0) = seq[0]
f(n) = max(seq[i], f(n-1))
```
Deve ser feita agora a definição da função em logica do SMT.
A função max é feita da seguinte forma:
$$max(a,b) \equiv Ite(a > b, a, b) \quad (\textrm{do mesmo modo:} \quad min(a,b) \equiv Ite(a < b, a, b))$$
A definição da função é feita, então da seguinte forma:
$$f(0) = seq[0] \land \\ 
\forall_n \quad 0 < n < N \rightarrow f(i-1) = Ite(seq[i] > f(n-1), seq[i], f(n-1))$$

Com o auxilio da função, a transição é feita da seguinte forma:
$$\forall_n \quad 0 \le n \lt N-1 \ \rightarrow seq'[i] = min(seq[i], f(i-1)) \land \\
q'[n-i] = f(n-1)$$

2. Criar n sequências e definir a transição através de várias atribuições.

Para tal definimos uma função python auxiliar, 'transaux(seq,seqlinha, I)' que atribui a $seqlinha[i]$, $min(seq[i],seq[i+1])$ e a $seqlinha[i+1]$, $max(seq[i],seq[i+1])$ mantendo a ordem do resto dos elementos.

Com a função anterior definida, a função de transição é feita criando uma lista de sequências,$l$ de forma a que $l[0] = seq$ e $l[-1] = seq'$ e retornar a disjunção da função 'transaux' aplicada a $l[i],l[i+1]\quad \forall_i: \quad 0<=i< N-1$

4-
Para provar a correção do algoritmo foi usado SAU com os algoritmos dados pelo professor.





## Código Python

Inicialmente apresentamos os algoritmos de SAU dados pelo professor.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *
import random as rn

def prove(f):
    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        if s.solve():
            print("Failed to prove.")
        else:
            print("Proved.")

In [2]:
# Auxiliares
def prime(v):
    return Symbol("next(%s)" % v.symbol_name(), v.symbol_type())
def fresh(v):
    return FreshSymbol(typename=v.symbol_type(),template=v.symbol_name()+"_%d")

# A classe "Sigle Assignment Unfold"
class SAU(object):
    """Trivial representation of a while cycle and its unfolding."""
    def __init__(self, variables, pre , pos, control, trans, sname="z3"):
              
        self.variables = variables       # variables   
        self.pre = pre                   # pre-condition as a predicate in "variables"
        self.pos = pos                   # pos-condition as a predicate in "variables"
        self.control = control           # cycle control as a predicate in "variables"
        self.trans = trans               # cycle body as a binary transition relation 
                                         # in "variables" and "prime variables"
        
        self.prime_variables = [prime(v) for v in self.variables]
        self.frames = [And([Not(control),pos])]  
                 # inializa com uma só frame: a da terminação do ciclo
        
        self.solver = Solver(name=sname)

    def new_frame(self):        
        freshs = [fresh(v) for v in self.variables]    
        b = self.control
        S = self.trans.substitute(dict(zip(self.prime_variables,freshs)))
        W = self.frames[-1].substitute(dict(zip(self.variables,freshs)))
        
        self.frames.append(And([b , ForAll(freshs, Implies(S, W))]))
        
    def unfold(self,bound=0):
        n = 0
        while True:
            if n > bound:
                print("falha: número de tentativas ultrapassa o limite %d "%bound)
                break
            
            f = Or(self.frames)
            if self.solver.solve([self.pre,Not(f)]):  
                self.new_frame()
                print(n)
                n += 1
            else:
                print("sucesso na tentativa %d "%n)
                break   

Nesta zona de código encontra-se a declaração de algumas variaveis bem como algumas condições de inicialização e a declaração da função, feita feita de forma semelhante ao exemplo da ficha 13.

In [9]:
N = rn.randint(20,50)

n = Int(N)

l = list()
for x in range(N):
    l.append(rn.randint(-50,51))
    
rn.shuffle(l)

seq = Symbol('seq', ArrayType(INT,INT))
i = Symbol('i', INT)
changed = Symbol('changed', BOOL)

store = And([Equals(Select(seq, Int(i)), Int(l[i])) for i in range(N)])


bubble_up = Symbol('bubble_up', FunctionType(INT,[INT]))
ax1 = Equals(bubble_up(Int(0)), Select(seq, Int(0)))
ax2 = ForAll([i], Implies(And(i>Int(0), i<n), Equals(bubble_up(i), Ite(Select(seq,i) >= bubble_up(i-Int(1)), Select(seq,i), bubble_up(i-Int(1))))))
axioms = And(ax1,ax2)




Aqui está definida a função de transição que faz uso da função auxiliar bubble_up.

In [4]:
def trans1(seq, seqlinha, changed, changedlinha):
    # print(seq,seqlinha)
    indutivo = ForAll([i], Implies(And(i<n-Int(1), i>=Int(0)), 
                                   Equals(Select(seqlinha,i), Ite(Select(seq, i+Int(1)) <= bubble_up(i), Select(seq, i+Int(1)), bubble_up(i)))))
    
    final = Equals(Select(seqlinha, n - Int(1)), bubble_up(n - Int(1)))
    
    
    troca_true = Iff(Iff(changedlinha, Bool(True)), Not(Equals(seq,seqlinha)))
    
    troca_false = Iff(Iff(changedlinha, Bool(False)), Equals(seq,seqlinha))
    
    return And(axioms,indutivo, troca_true, troca_false, final)
    
     
    
    

Aqui está definida a função de transição que cria n sequências.

In [5]:
def transaux(seq,seqlinha,I):
    i = Int(I)
    l = list()
    for n in range(N):
        l.append(Equals(Select(seqlinha,Int(n)), Select(seq,Int(n))))# copia o seq para o seqlinha
    
    l[I] = Equals(Select(seqlinha, i), Ite(Select(seq,i) < Select(seq, i+Int(1)), Select(seq,i), Select(seq, i+Int(1))))
    l[I+1] = Equals(Select(seqlinha, i+Int(1)), Ite(Select(seq,i) > Select(seq, i+Int(1)), Select(seq,i), Select(seq, i+Int(1))))

    return And(l)
    

def trans2(seq,seq_p,changed,changed_p):
    seqlist = []
    for i in range(N):
        seqlist.append(Symbol('seq' + str(i), ArrayType(INT,INT)))
    seqlist[0] = seq
    seqlist[-1] = seq_p
    
    troca_true = Iff(Iff(changed_p, Bool(True)), Not(Equals(seq,seq_p)))
    
    troca_false = Iff(Iff(changed_p, Bool(False)), Equals(seq,seq_p))
        
    return And(And([transaux(seqlist[i],seqlist[i+1],i) for i in range(N-1)]),troca_true,troca_false)

Aqui são definidas a pré e pós condição, a condiçao de controlo do ciclo e a condição de transição.
Além disso é feita a prova da correção do programa.

In [6]:

variables = [seq,changed]

pre  =  And(n>=Int(0), Iff(changed, Bool(True)), store) 
pos  =  And(ForAll([i], Implies(And(i>=0, i<n-Int(1)), Select(seq, i) <= Select(seq, i+Int(1)))), Iff(changed, Bool(False)))
cond =  Iff(changed, Bool(True))               # condição de controlo do ciclo
trans = trans1(seq, prime(seq), changed, prime(changed))

#trans = trans_seq(seq,prime(seq),changed,prime(changed))

## Conclusão
Após vários testes, especialmente para comprimentos de lista grandes, notamos que a função 'trans1' não tem sempre o comportamento experado e tende a fazer com que o solver retorne 'UnknownResultError' embora este após a reexecução do programa a mensagen de erro não apareça, por isso decidimos que a maioria dos testes apresentados fariam uso da função trans2.
Além disso notamos que em todos os casos testados o método unfold teve sucesso na segunda tentativa.

In [7]:
print(N)
print(l)

W = SAU(variables,pre,pos,cond,trans)

W.unfold(N)

41
[-43, 36, -10, -27, -34, 4, -33, -30, -17, -32, -41, -43, 44, -34, -16, 23, -14, -26, 33, -50, 45, -10, 36, 43, -39, -25, -43, 48, 13, 51, 38, 2, 29, 18, -36, -39, -27, 13, -23, -50, -5]
0
1
sucesso na tentativa 2 


In [8]:
print(N)
print(l)

W = SAU(variables,pre,pos,cond,trans)

W.unfold(N)

41
[-43, 36, -10, -27, -34, 4, -33, -30, -17, -32, -41, -43, 44, -34, -16, 23, -14, -26, 33, -50, 45, -10, 36, 43, -39, -25, -43, 48, 13, 51, 38, 2, 29, 18, -36, -39, -27, 13, -23, -50, -5]
0
1
sucesso na tentativa 2 


In [9]:
print(N)
print(l)

W = SAU(variables,pre,pos,cond,trans)

W.unfold(N)

41
[-43, 36, -10, -27, -34, 4, -33, -30, -17, -32, -41, -43, 44, -34, -16, 23, -14, -26, 33, -50, 45, -10, 36, 43, -39, -25, -43, 48, 13, 51, 38, 2, 29, 18, -36, -39, -27, 13, -23, -50, -5]
0
1
sucesso na tentativa 2 


In [10]:
print(N)
print(l)

W = SAU(variables,pre,pos,cond,trans)

W.unfold(N)

41
[-43, 36, -10, -27, -34, 4, -33, -30, -17, -32, -41, -43, 44, -34, -16, 23, -14, -26, 33, -50, 45, -10, 36, 43, -39, -25, -43, 48, 13, 51, 38, 2, 29, 18, -36, -39, -27, 13, -23, -50, -5]
0
1
sucesso na tentativa 2 


Os exemplos seguintes foram executados com 'trans1' 

In [7]:
print(N)
print(l)

W = SAU(variables,pre,pos,cond,trans)

W.unfold(N)

32
[30, -25, 51, -24, -13, 23, 12, 20, -33, 34, 19, -8, -3, -12, -36, 41, -39, 11, -48, 24, 36, -18, -6, 49, -44, -6, 30, 22, -21, -46, 34, -11]
0
1
sucesso na tentativa 2 


In [7]:
print(N)
print(l)

W = SAU(variables,pre,pos,cond,trans)

W.unfold(N)

38
[-17, -2, 10, -41, 18, 47, 3, 30, -46, -43, -39, 25, 26, 43, 13, 33, 15, -16, -8, -18, -38, -41, -25, -10, -9, -12, -30, -11, -22, 22, -35, 51, 41, 1, 14, -7, -10, 50]
0
1
sucesso na tentativa 2 


In [7]:
print(N)
print(l)

W = SAU(variables,pre,pos,cond,trans)

W.unfold(N)

50
[33, 25, -15, 13, 48, 31, -9, -40, 45, -20, -18, -40, 18, 35, 2, -40, 31, 14, 15, 46, 39, -17, 0, 25, 44, 31, 4, 40, 20, -42, 33, 7, -35, -8, 13, 3, -44, 33, 44, -14, 35, 15, -27, 40, 36, 18, -6, -35, -19, 26]
0
1
sucesso na tentativa 2 
